In [2]:
#IMPORT LIBRARIES
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [4]:
#unzip the file, super large csv
import zipfile
with zipfile.ZipFile("datasets/airlinereviews128k.zip", 'r') as zip_ref:
    zip_ref.extractall("datasets")

In [5]:
#Import the data
df0=pd.read_csv("datasets/airlinereviews128k.csv", parse_dates=['DateFlown'])

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
df0.sample()

,Aircraft,AirlineName,CabinType,DateFlown,DatePub,EntertainmentRating,FoodRating,GroundServiceRating,OriginCountry,OverallScore,Recommended,Review,Route,SeatComfortRating,ServiceRating,Slug,Title,TravelType,TripVerified,ValueRating,WifiRating,unique_id
35201,NaN,China Eastern Airlines,Economy Class,2017-06-01,5th June 2017,1,1,2,UnitedStates,1.0,no,Shanghai to Hong Kong with China Eastern Airli...,Shanghai to Hong Kong,4,1,china-eastern-airlines,"asked for a blanket, instantly told they were out",Business,NaN,4,0,e59f74f2-28d0-4467-ad9e-e2cc653e4f4d


In [8]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129455 entries, 0 to 129454
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Aircraft             36423 non-null   object        
 1   AirlineName          129455 non-null  object        
 2   CabinType            126437 non-null  object        
 3   DateFlown            90993 non-null   datetime64[ns]
 4   DatePub              129455 non-null  object        
 5   EntertainmentRating  129455 non-null  int64         
 6   FoodRating           129455 non-null  int64         
 7   GroundServiceRating  129455 non-null  int64         
 8   OriginCountry        127777 non-null  object        
 9   OverallScore         125124 non-null  float64       
 10  Recommended          129455 non-null  object        
 11  Review               128631 non-null  object        
 12  Route                90825 non-null   object        
 13  SeatComfortRat

## EDA

In [6]:
#pass names to uppercase (to differentiate better)
df0.columns = [x.upper() for x in df0.columns]

In [7]:
#keep only dataset with Review values and Title of Reviews
df0=df0.dropna(subset="REVIEW") #too complicated to impute
df0=df0.dropna(subset="TITLE") #too complicated to impute
df0.reset_index(drop=True, inplace=True)

In [8]:
df0.ROUTE.unique() #probably transform it later for origin and destiny

array(['Moroni to Moheli', 'Moroni to Anjouan', 'Anjouan to Dzaoudzi',
       ..., 'Budapest to Göteborg', 'London to Burgas',
       'Paris Beauvais to Riga'], dtype=object)

In [9]:
#clean the tripverified variable
df0.TRIPVERIFIED.value_counts()

TRIPVERIFIED
Trip Verified                                45311
Not Verified                                 13755
NotVerified                                      9
Trip Verified,Trip Verified                      1
Chicago to Colorado Springs. Not Verified        1
Name: count, dtype: int64

In [10]:
df0["TRIPVERIFIED"].replace("Trip Verified,Trip Verified","Trip Verified ", inplace=True)
df0["TRIPVERIFIED"].replace("NotVerified","Not Verified ", inplace=True)
df0["TRIPVERIFIED"].replace("Not Verified,Not Verified","Not Verified ", inplace=True)
df0["TRIPVERIFIED"].replace("Chicago to Colorado Springs. Not Verified","Not Verified ", inplace=True)

df0["TRIPVERIFIED"]=df0.TRIPVERIFIED.str.strip()

In [11]:
#check time horizon to see if enough for time series
print(df0["DATEPUB"].min())
print(df0["DATEPUB"].max())

#ok

10th April 2005
9th September 2022


In [12]:
df0.drop_duplicates()
df0.shape #no duplicates

(128627, 22)

In [13]:
df0.describe()

,DATEFLOWN,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,OVERALLSCORE,SEATCOMFORTRATING,SERVICERATING,VALUERATING,WIFIRATING
count,90166,128627.000000,128627.000000,128627.000000,124297.000000,128627.000000,128627.000000,128627.000000,128627.000000
mean,2018-12-26 22:17:51.144333568,1.559260,1.991557,1.648464,4.563103,2.471518,2.690602,2.639780,0.453132
min,2012-04-01 00:00:00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2017-01-01 00:00:00,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,2018-11-01 00:00:00,1.000000,1.000000,1.000000,3.000000,2.000000,3.000000,2.000000,0.000000
75%,2020-11-01 00:00:00,3.000000,4.000000,3.000000,8.000000,4.000000,4.000000,4.000000,0.000000
max,2023-05-01 00:00:00,5.000000,5.000000,5.000000,10.000000,5.000000,5.000000,5.000000,5.000000
std,NaN,1.751718,1.774979,1.740740,3.499330,1.620989,1.775078,1.629001,1.077762


In [14]:
from functions import nulls

In [15]:
#function to check missingness
nulls(df0) #most under 3% so ok to impute. Check also if some variables have 0s where missing. So far: assume when 0 that is really poor service (except entertainment, that means short-distance flight)

,column_name,nulls_percentage
0,AIRCRAFT,0.718939
1,AIRLINENAME,0.000000
2,CABINTYPE,0.023455
3,DATEFLOWN,0.299012
4,DATEPUB,0.000000
5,ENTERTAINMENTRATING,0.000000
6,FOODRATING,0.000000
7,GROUNDSERVICERATING,0.000000
8,ORIGINCOUNTRY,0.013045
9,OVERALLSCORE,0.033663


Need for imputation:
- AIRCRAFT:won't be used so far, keep it like this for now
- TRAVELTYPE: drop values as I want to keep this var but too many missing to impute efficiently
- DATEFLOWN: ok, impute
- ROUTE: ok, impute
- CABINTYPE: ok, impute
- OVERALLSCORE: ok, impute

In [16]:
#too many missing on traveltype but I want that variable so let's drop null values from it (still enough observ.)

df0=df0.dropna(subset="TRAVELTYPE")

In [17]:
df0.DATEFLOWN.isna().sum() #ok

158

In [18]:
pd.Timestamp.min
df0['DATEFLOWN']=pd.to_datetime(df0['DATEFLOWN'].fillna(pd.Timestamp.min))

In [19]:
#impute using the most common route (mode)
df0['ROUTE'] = df0['ROUTE'].fillna(df0['ROUTE'].mode()[0])

In [20]:
df0.ROUTE.isna().sum() #ok

0

In [21]:
df0['CABINTYPE'] = df0['CABINTYPE'].fillna(df0['CABINTYPE'].mode()[0])

In [22]:
df0.CABINTYPE.isna().sum() #ok

0

In [23]:
nulls(df0)

,column_name,nulls_percentage
0,AIRCRAFT,0.599763
1,AIRLINENAME,0.000000
2,CABINTYPE,0.000000
3,DATEFLOWN,0.000000
4,DATEPUB,0.000000
5,ENTERTAINMENTRATING,0.000000
6,FOODRATING,0.000000
7,GROUNDSERVICERATING,0.000000
8,ORIGINCOUNTRY,0.000000
9,OVERALLSCORE,0.000011


In [24]:
df0.OVERALLSCORE.value_counts()

OVERALLSCORE
1.0     39008
2.0      9363
10.0     9012
9.0      7413
8.0      6775
3.0      5541
7.0      4459
4.0      3099
5.0      2964
6.0      2684
Name: count, dtype: int64

In [25]:
df0['OVERALLSCORE']=df0.OVERALLSCORE.fillna(round(np.mean(df0.OVERALLSCORE))) 

In [26]:
df0.OVERALLSCORE.isna().sum() #ok

0

In [27]:
#TripVerified: assume that missing is 'not verified'

In [28]:
df0['TRIPVERIFIED']=df0.TRIPVERIFIED.fillna("Not Verified") 

In [29]:
df0.TRIPVERIFIED.isna().sum() #ok

0

In [30]:
#for now, in the rating vars asumme that 0 is 0 and non missing (if no service, I put a 0)

In [31]:
df0["OVERALLSCORE"] = df0["OVERALLSCORE"].astype(int)

In [32]:
df0.reset_index(drop=True, inplace=True)

In [33]:
df=df0.drop(columns="AIRCRAFT")

In [34]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AIRCRAFT             36149 non-null  object        
 1   AIRLINENAME          90319 non-null  object        
 2   CABINTYPE            90319 non-null  object        
 3   DATEFLOWN            90319 non-null  datetime64[ns]
 4   DATEPUB              90319 non-null  object        
 5   ENTERTAINMENTRATING  90319 non-null  int64         
 6   FOODRATING           90319 non-null  int64         
 7   GROUNDSERVICERATING  90319 non-null  int64         
 8   ORIGINCOUNTRY        90319 non-null  object        
 9   OVERALLSCORE         90319 non-null  int32         
 10  RECOMMENDED          90319 non-null  object        
 11  REVIEW               90319 non-null  object        
 12  ROUTE                90319 non-null  object        
 13  SEATCOMFORTRATING    90319 non-

In [35]:
df0.shape

(90319, 22)

In [36]:
df.shape 

(90319, 21)

In [37]:
##CHECK VALUES OF VARIABLES

In [38]:
df.CABINTYPE.value_counts() #ok

CABINTYPE
Economy Class      74836
Business Class     10507
Premium Economy     3360
First Class         1616
Name: count, dtype: int64

In [39]:
df.ENTERTAINMENTRATING.value_counts()

ENTERTAINMENTRATING
0    39937
1    19423
4     8641
3     8480
5     7984
2     5854
Name: count, dtype: int64

In [40]:
df.FOODRATING.value_counts()

FOODRATING
0    26019
1    23418
5    10827
3    10675
4    10594
2     8786
Name: count, dtype: int64

In [41]:
df.GROUNDSERVICERATING.value_counts()

GROUNDSERVICERATING
1    42006
5    15405
4    12225
3     9399
2     7759
0     3525
Name: count, dtype: int64

In [42]:
df.OVERALLSCORE.value_counts()

OVERALLSCORE
1     39008
2      9363
10     9012
9      7413
8      6775
3      5541
7      4459
4      3100
5      2964
6      2684
Name: count, dtype: int64

In [43]:
df.SEATCOMFORTRATING.value_counts()

SEATCOMFORTRATING
1    28506
3    15943
4    14855
2    12681
5    12141
0     6193
Name: count, dtype: int64

In [44]:
df.SERVICERATING.value_counts()

SERVICERATING
1    26623
5    21438
4    12694
3    12484
2    10587
0     6493
Name: count, dtype: int64

In [45]:
df.VALUERATING.value_counts()

VALUERATING
1    41477
5    17176
4    11373
2    10647
3     9632
0       14
Name: count, dtype: int64

In [46]:
df.WIFIRATING.value_counts()

WIFIRATING
0    61095
1    17846
5     3137
3     3048
4     2612
2     2581
Name: count, dtype: int64

In [47]:
df.TRAVELTYPE.value_counts()

TRAVELTYPE
Solo Leisure      32957
Couple Leisure    23611
Family Leisure    19703
Business          14048
Name: count, dtype: int64

In [48]:
df.TRIPVERIFIED.value_counts()

TRIPVERIFIED
Trip Verified    45310
Not Verified     45009
Name: count, dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AIRLINENAME          90319 non-null  object        
 1   CABINTYPE            90319 non-null  object        
 2   DATEFLOWN            90319 non-null  datetime64[ns]
 3   DATEPUB              90319 non-null  object        
 4   ENTERTAINMENTRATING  90319 non-null  int64         
 5   FOODRATING           90319 non-null  int64         
 6   GROUNDSERVICERATING  90319 non-null  int64         
 7   ORIGINCOUNTRY        90319 non-null  object        
 8   OVERALLSCORE         90319 non-null  int32         
 9   RECOMMENDED          90319 non-null  object        
 10  REVIEW               90319 non-null  object        
 11  ROUTE                90319 non-null  object        
 12  SEATCOMFORTRATING    90319 non-null  int64         
 13  SERVICERATING        90319 non-

## FEATURE ENGINEERING

In [50]:
#Transform the travel type variable in two: type of traveller (solo, family, business) and purpose (leisure, business)

In [51]:
travell_transf=df["TRAVELTYPE"].str.split(" ", expand=True)

In [52]:
travell_transf.rename(columns={travell_transf.columns[0]: 'TRAVELLER'}, inplace=True)
travell_transf.rename(columns={travell_transf.columns[1]: 'PURPOSE'}, inplace=True)

In [53]:
travell_transf.TRAVELLER.value_counts()

TRAVELLER
Solo        32957
Couple      23611
Family      19703
Business    14048
Name: count, dtype: int64

In [54]:
travell_transf["PURPOSE"]=np.where(travell_transf['TRAVELLER']=="Business", "Bussiness",travell_transf.PURPOSE)

In [55]:
travell_transf["TRAVELLER"]=np.where(travell_transf['TRAVELLER']=="Business", "Solo",travell_transf.TRAVELLER) #assume business is 'solo' in traveller_type

In [56]:
df.columns.get_loc('TRAVELTYPE') 

16

In [57]:
df = pd.concat([df.iloc[:, :17], travell_transf, df.iloc[:, 17:]], axis=1)

In [58]:
df.drop(columns='TRAVELTYPE', inplace=True)

In [59]:
df.RECOMMENDED.value_counts() #quite imbalanced. Need to balance it?

RECOMMENDED
no     59309
yes    31010
Name: count, dtype: int64

In [60]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
66493,Southwest Airlines,Economy Class,2022-09-01,26th September 2022,1,1,1,UnitedStates,1,no,I had two carry on bags and a cross body bag (...,Seattle to Sacramento,2,3,southwest-airlines,paid $30 extra dollars to get priority boarding,Solo,Leisure,Not Verified,1,1,8011588c-e106-478d-8647-39554f92abf3


In [61]:
#clean variable #ROUTE

In [62]:
route_origin = df["ROUTE"].str.split(r"\bto\b", expand=True)

In [63]:
route_origin.head()

,0,1,2,3,4
0,Moroni,Moheli,None,None,None
1,Moroni,Anjouan,None,None,None
2,Anjouan,Dzaoudzi,None,None,None
3,Milano,Olbia,None,None,None
4,Cascais,Bragança,None,None,None


In [64]:
route_origin.reset_index(drop=True, inplace=True)

In [65]:
route_origin.info() #will assume that other 'to' are not necessary (writing typos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       90319 non-null  object
 1   1       90113 non-null  object
 2   2       57 non-null     object
 3   3       6 non-null      object
 4   4       1 non-null      object
dtypes: object(5)
memory usage: 3.4+ MB


In [66]:
df.ROUTE.unique() #clearly origin and destiny separated by 'to'

array(['Moroni to Moheli', 'Moroni to Anjouan', 'Anjouan to Dzaoudzi',
       ..., 'Budapest to Göteborg', 'London to Burgas',
       'Paris Beauvais to Riga'], dtype=object)

In [67]:
route_origin.reset_index(drop=True, inplace=True)

In [68]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
15806,American Airlines,Economy Class,2019-08-01,19th August 2019,0,1,1,UnitedStates,1,no,Phoenix to Houston IAH. My back still hurts fr...,Phoenix to Houston IAH,1,2,american-airlines,most cramped seats,Solo,Leisure,Not Verified,1,0,c4db2959-dc76-436c-9ee6-c06884a8cff0


In [69]:
df.reset_index(drop=True, inplace=True) #important to reset index or won't fit the new variables with route

In [70]:
df.columns.get_loc('ROUTE')

11

In [71]:
df = pd.concat([df.iloc[:, :12], route_origin, df.iloc[:, 12:]], axis=1)

In [72]:
pd.set_option('display.max_columns', None) #to check all columns when sampling the df

In [73]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,0,1,2,3,4,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
30763,Etihad Airways,Economy Class,2019-03-01,19th March 2019,4,4,2,UnitedKingdom,8,yes,Mumbai to London via Abu Dhabi. I can only sur...,Mumbai to London via Abu Dhabi,Mumbai,London via Abu Dhabi,None,None,None,4,4,etihad-airways,Emirates and Qatar do a better job,Couple,Leisure,Trip Verified,5,1,fe64e189-3169-48bd-aeb4-de1e9f160ba8


In [74]:
df.drop(columns=df.columns[15], inplace=True) #3 times, to take the extra 'tos', we are goint to assume there is only a 'to'

In [75]:
df.drop(columns=df.columns[15], inplace=True) 

In [76]:
df.drop(columns=df.columns[14], inplace=True) 

In [77]:
df.rename(columns={df.columns[12]: 'ORIGIN'}, inplace=True)
df.rename(columns={df.columns[13]: 'DESTINY'}, inplace=True)

In [78]:

df.reset_index(drop=True, inplace=True)

In [79]:
route_destiny = df["DESTINY"].str.split(r"\bvia\b", expand=True)

In [80]:
route_destiny.sample() #if scale is 'none' this means no scale was done. 

,0,1,2
64977,Dar es Salaam,None,None


In [81]:
route_destiny.rename(columns={route_destiny.columns[0]: 'DESTINY2'}, inplace=True)
route_destiny.rename(columns={route_destiny.columns[1]: 'SCALE'}, inplace=True)
route_destiny.rename(columns={route_destiny.columns[2]: 'scale2'}, inplace=True)

In [82]:
route_destiny.scale2.value_counts() #very few, we assume only if scale or not

scale2
Paris          1
HKG            1
Mexico City    1
Name: count, dtype: int64

In [83]:
route_destiny.drop(columns="scale2", inplace=True)

In [84]:
route_destiny.reset_index(drop=True, inplace=True)

In [85]:
df.columns.get_loc('DESTINY')

13

In [86]:
df = pd.concat([df.iloc[:, :14], route_destiny, df.iloc[:, 14:]], axis=1)

In [87]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,ORIGIN,DESTINY,DESTINY2,SCALE,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
53585,Lufthansa,Economy Class,2016-10-01,9th October 2016,2,2,1,Germany,3,no,"✅ , | Once more the Lufthansa bus from Strasb...",SXB to ATH via FRA,SXB,ATH via FRA,ATH,FRA,2,4,lufthansa,LH is really bad in timeliness,Solo,Bussiness,Not Verified,3,3,14dc76e1-0471-4c57-9747-f2f36b1d4287


In [88]:
df.drop(columns="DESTINY", inplace=True)

In [89]:
df.rename(columns={'DESTINY2':'DESTINY'}, inplace=True)

In [90]:
df.reset_index(drop=True, inplace=True) 

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90319 entries, 0 to 90318
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AIRLINENAME          90319 non-null  object        
 1   CABINTYPE            90319 non-null  object        
 2   DATEFLOWN            90319 non-null  datetime64[ns]
 3   DATEPUB              90319 non-null  object        
 4   ENTERTAINMENTRATING  90319 non-null  int64         
 5   FOODRATING           90319 non-null  int64         
 6   GROUNDSERVICERATING  90319 non-null  int64         
 7   ORIGINCOUNTRY        90319 non-null  object        
 8   OVERALLSCORE         90319 non-null  int32         
 9   RECOMMENDED          90319 non-null  object        
 10  REVIEW               90319 non-null  object        
 11  ROUTE                90319 non-null  object        
 12  ORIGIN               90319 non-null  object        
 13  DESTINY              90113 non-

In [92]:
df= df.dropna(axis=0, subset=['DESTINY']) # too complicated to clean and separate, so drop (only 37 values)

In [93]:
#delete spaces before and after
df["ORIGIN"]=df.ORIGIN.str.strip()
df["DESTINY"]=df.DESTINY.str.strip()

In [94]:
df.insert(15,'SCALE_YN',np.where(df['SCALE'].isna(), 0,1))

In [95]:
df.drop(columns="SCALE", inplace=True)

In [96]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,ORIGIN,DESTINY,SCALE_YN,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID
44677,Jetblue Airways,Economy Class,2021-06-01,6th June 2021,1,1,1,UnitedStates,1,no,I bought 4 tickets from Orlando to Jamaica (1s...,Orlando to Montego Bay,Orlando,Montego Bay,0,2,1,jetblue-airways,has the worst customer service,Family,Leisure,Trip Verified,1,1,a25af882-aa05-474c-9062-8b9124a9b9e0


In [97]:
#create a variable that states whether the flight was overseas or not. If entertainment diff. from 0 -->overseas; otherwise, no.
df['LONGDISTANCE']=np.where(df.ENTERTAINMENTRATING==0,0,1)

In [98]:
df['TRIPVERIFIED']=np.where(df['TRIPVERIFIED']=="Trip Verified", 1,0) 

In [99]:
df.RECOMMENDED.value_counts() #quite imbalanced...better check later balancing with SMOTE?

RECOMMENDED
no     59224
yes    30889
Name: count, dtype: int64

In [100]:
df_categ=df.copy()

In [101]:
df_categ=df_categ.to_excel(("datasets/categoricalclean.xlsx"))

In [102]:
#pass RECOMMENDED TO DUMMY
df["RECOMMENDED"]=np.where(df["RECOMMENDED"]=="yes",1,0)

In [103]:
#change categoricals string to categorical numbers 1=lowest class and 4=highest class
df['CABINTYPE']=np.where(df['CABINTYPE']=="Economy Class", 1,df.CABINTYPE)
df['CABINTYPE']=np.where(df['CABINTYPE']=="Premium Economy", 2,df.CABINTYPE)
df['CABINTYPE']=np.where(df['CABINTYPE']=="Business Class", 3,df.CABINTYPE) #in order to avoid such an inbalanced class (firstclass few n)
df['CABINTYPE']=np.where(df['CABINTYPE']=="First Class", 4,df.CABINTYPE)

In [104]:
#pass froms string to int
df["CABINTYPE"]=df["CABINTYPE"].astype(int)

In [105]:
df.TRAVELLER.value_counts()

TRAVELLER
Solo      46871
Couple    23569
Family    19673
Name: count, dtype: int64

In [106]:
#change the variable traveller to numeric (based on specific order)
df['TRAVELLER']=np.where(df['TRAVELLER']=="Solo", 1,df.TRAVELLER)
df['TRAVELLER']=np.where(df['TRAVELLER']=="Couple", 2,df.TRAVELLER)
df['TRAVELLER']=np.where(df['TRAVELLER']=="Family", 3,df.TRAVELLER)

In [107]:
#pass froms string to int
df["TRAVELLER"]=df["TRAVELLER"].astype(int)

In [108]:
df.PURPOSE.value_counts()

PURPOSE
Leisure      76103
Bussiness    14010
Name: count, dtype: int64

In [109]:
#change the variable traveller to numeric (based on specific order)
df['PURPOSE']=np.where(df['PURPOSE']=="Leisure", 1,0) 

In [110]:
df.sample()

,AIRLINENAME,CABINTYPE,DATEFLOWN,DATEPUB,ENTERTAINMENTRATING,FOODRATING,GROUNDSERVICERATING,ORIGINCOUNTRY,OVERALLSCORE,RECOMMENDED,REVIEW,ROUTE,ORIGIN,DESTINY,SCALE_YN,SEATCOMFORTRATING,SERVICERATING,SLUG,TITLE,TRAVELLER,PURPOSE,TRIPVERIFIED,VALUERATING,WIFIRATING,UNIQUE_ID,LONGDISTANCE
88804,WOW air,1,2018-12-01,20th December 2018,1,0,1,UnitedStates,1,0,Los Angeles to Berlin via Keflavik. I had a ve...,Los Angeles to Berlin via Keflavik,Los Angeles,Berlin,1,2,5,wow-air,very unpleasant experience,1,1,0,3,0,e642335a-a782-4d26-a6d8-8c9b6bafb8e7,1


In [111]:
df['YEAR'] = df['DATEFLOWN'].dt.year

In [112]:
df.shape

(90113, 27)

In [113]:
df.YEAR.value_counts() #1677 makes no sense! drop these 105 values

YEAR
2019    15199
2018    14040
2022    12142
2016    11847
2017    11481
2015     9734
2021     6329
2020     5529
2023     3557
2014      147
1677      105
2013        2
2012        1
Name: count, dtype: int64

In [114]:
df = df[(df.YEAR > 1677)] #avoid negative values --> 1677

In [115]:
##CLEAN DATABASE!
df.shape

(90008, 27)

In [116]:
#save clean data
df_eda_clean=df.to_csv("datasets/cleandf.csv")
df_eda_clean=df.to_csv("datasets/streamlitdb/cleandf.csv")

In [117]:
df_eda_clean=df.copy()